This code fetches the info of all the teams regardless of league etc.

It then fetches the players on these teams.

Then every player in S14.

This only needs to be run once(per season)

In [1]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd

GAME_INFO_BASE_URL = 'https://gol.gg/teams/list/season-ALL/split-ALL/tournament-ALL/'
GAME_INFO_HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/117.0"
}


In [2]:
page = requests.get(GAME_INFO_BASE_URL, headers=GAME_INFO_HEADERS)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
main_table = soup.find_all('table', class_ = 'table_list')[0]

In [4]:
df = pd.DataFrame(columns=['team_id'])

for col in main_table.find('thead').find_all('th'):
    name = col.text.strip()
    df[name] = ''

df.head()


,team_id,Name,Season,Region,Games,Win rate,K:D,GPM,GDM,Game duration,...,TD@15,GD@15,PPG,NASHPG,NASH%,CSM,DPM,WPM,VWPM,WCPM


In [5]:
def get_team_id(info):
    print(info)
    return info.find('a').get('href').split('/')[2]

In [6]:
for row in main_table.find_all('tr')[1:]:
    new_team = dict()
    for i,info in enumerate(row.find_all('td')):
        if i == 0:
            new_team[df.columns[0]] = get_team_id(info)
            new_team[df.columns[1]] = info.text
        else:
            new_team[df.columns[i+1]] = info.text

    df = pd.concat([df, pd.DataFrame([new_team])], ignore_index=True)


<td><a href="./team-stats/2146/split-ALL/tournament-ALL/" title="100 Thieves stats">100 Thieves</a></td>
<td><a href="./team-stats/2407/split-ALL/tournament-ALL/" title="A One Man Army stats">A One Man Army</a></td>
<td><a href="./team-stats/2229/split-ALL/tournament-ALL/" title="Aegis stats">Aegis</a></td>
<td><a href="./team-stats/2231/split-ALL/tournament-ALL/" title="Akroma stats">Akroma</a></td>
<td><a href="./team-stats/2397/split-ALL/tournament-ALL/" title="All for One Gaming stats">All for One Gaming</a></td>
<td><a href="./team-stats/2216/split-ALL/tournament-ALL/" title="All Knights stats">All Knights</a></td>
<td><a href="./team-stats/2245/split-ALL/tournament-ALL/" title="aNc Outplayed stats">aNc Outplayed</a></td>
<td><a href="./team-stats/2312/split-ALL/tournament-ALL/" title="Ankora Gaming stats">Ankora Gaming</a></td>
<td><a href="./team-stats/2302/split-ALL/tournament-ALL/" title="Anorthosis Famagusta Esports stats">Anorthosis Famagusta Esports</a></td>
<td><a href="./

In [2]:
df.to_csv('S14_GENERAL_TEAM_INFO.csv',index=False)

NameError: name 'df' is not defined

Fetching players on each team here

In [88]:
df_players_team = pd.DataFrame(columns=['top','jungle','mid','bot','support'])

In [90]:
for team_id in df['team_id']:

    team_url = team_stats_url.format(team_id)
    page = requests.get(team_url, headers=GAME_INFO_HEADERS)
    new_soup = BeautifulSoup(page.content, 'html.parser')

    new_team = dict()
    players_counted = 0
    for info in new_soup.find_all('table',class_ = 'table_list')[-1].find_all('tr')[1:]:
        if len(info.find_all('td')) == 1:
            continue

        pos = info.find_all('td')[0].text.lower().strip()
        player = info.find_all('td')[1].text.strip()
        if players_counted < 5:
            new_team[pos] = player
        else:
            new_team[F'sub_{players_counted % 5}_{pos}'] = player

        players_counted = players_counted + 1
    print(new_team)
    df_players_team = pd.concat([df_players_team, pd.DataFrame([new_team])], ignore_index=True)

{'top': 'Sniper', 'jungle': 'River', 'mid': 'Quid', 'bot': 'Meech', 'support': 'Eyla'}
{'top': 'W1ND', 'jungle': 'Sawyor', 'mid': 'pump', 'bot': 'Kehvo', 'support': 'B Butcher', 'sub_0_top': 'Zhergoth'}
{'top': 'Agresivoo', 'jungle': 'Ryuzaki', 'mid': 'Nafkelah', 'bot': 'Hid0', 'support': 'Veignorem'}
{'top': 'Theoloris', 'jungle': 'Viking', 'mid': 'Zhangsid', 'bot': 'Mishigu', 'support': 'Abagnale', 'sub_0_jungle': 'Xeonerr', 'sub_1_support': 'Velocity'}
{'top': 'Obelisk', 'jungle': 'dibu', 'mid': 'UniqueCORN', 'bot': 'Notiko', 'support': 'Ephekles', 'sub_0_mid': 'H0NEST'}
{'top': 'Hidan', 'jungle': 'Grell', 'mid': 'Nobody', 'bot': 'Lac', 'support': 'Gastruks', 'sub_0_bot': 'Trigo'}
{'top': 'Matixx', 'jungle': 'nicolaiy', 'mid': 'Vigil', 'bot': 'Orion', 'support': 'SHREDDER'}
{'top': 'YoOlek', 'jungle': 'Yuta', 'mid': 'Hunt', 'bot': 'Paladin', 'support': 'minemaciek', 'sub_0_top': 'Nazgul', 'sub_1_jungle': 'Velja'}
{'top': 'PapiTeero', 'jungle': 'Stormax', 'mid': 'Furuy', 'bot': 'gold

In [140]:
df_players_team.to_csv('s14_PLAYERS_TEAMS.csv',index=False)

GETTING PLAYER INFO

In [96]:
GAME_INFO_BASE_URL = 'https://gol.gg/players/list/season-S14/split-ALL/tournament-ALL/'

In [97]:
page = requests.get(GAME_INFO_BASE_URL, headers=GAME_INFO_HEADERS)
soup = BeautifulSoup(page.content, 'html.parser')

In [104]:
main_table = soup.find('table',class_ = 'table_list')

In [105]:
main_table.find('thead')

<thead><tr>
<th data-tablesaw-priority="persist" data-tablesaw-sortable-col="">Player</th>
<th data-tablesaw-sortable-col="">Country</th>
<th data-sortable-numeric="" data-tablesaw-sortable-col="">Games</th>
<th data-sortable-numeric="" data-tablesaw-sortable-col="">Win rate</th>
<th data-sortable-numeric="" data-tablesaw-sortable-col="">KDA</th>
<th data-sortable-numeric="" data-tablesaw-sortable-col="">Avg kills</th>
<th data-sortable-numeric="" data-tablesaw-sortable-col="">Avg deaths</th>
<th data-sortable-numeric="" data-tablesaw-sortable-col="">Avg assists</th>
<th data-placement="top" data-sortable-numeric="" data-tablesaw-sortable-col="" data-toggle="tooltip" title="Average CS per minute per game">CSM</th>
<th data-placement="top" data-sortable-numeric="" data-tablesaw-sortable-col="" data-toggle="tooltip" title="Average Gold per minute game">GPM</th>
<th data-placement="top" data-sortable-numeric="" data-tablesaw-sortable-col="" data-toggle="tooltip" title="Average Kill Partic

In [134]:
df_players = pd.DataFrame()

for col in main_table.find('thead').find_all('th'):
    name = col.text.strip().lower()
    df_players[name] = ''

df_players.head()


,player,country,games,win rate,kda,avg kills,avg deaths,avg assists,csm,gpm,...,avg wpm,avg wcpm,avg vwpm,gd@15,csd@15,xpd@15,fb %,fb victim,penta kills,solo kills


In [135]:
for row in main_table.find_all('tr')[1:]:
    new_player = dict()
    for col,info in zip(df_players.columns,row.find_all('td')):
        if '%' in info.text:
            new_player[col] =  float(info.text.strip()[:-1])
        elif info.text.strip() == '-':
            new_player[col] =  0
        elif col == 'player' or col == 'country':
            new_player[col] =  info.text.strip()
        else:    
            new_player[col] =  float(info.text.strip())

    df_players = pd.concat([df_players, pd.DataFrame([new_player])], ignore_index=True)


C:\Users\joshu_rdnqgbx\AppData\Local\Temp\ipykernel_16504\3047145650.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_players = pd.concat([df_players, pd.DataFrame([new_player])], ignore_index=True)


In [137]:
df_players

,player,country,games,win rate,kda,avg kills,avg deaths,avg assists,csm,gpm,...,avg wpm,avg wcpm,avg vwpm,gd@15,csd@15,xpd@15,fb %,fb victim,penta kills,solo kills
0,0909,CN,5.0,40.0,3.4,2.2,2.2,5.2,8.2,364.0,...,0,0,0,0,0,0,0,0,0,0
1,0din,CN,11.0,36.4,2.3,2.5,3.9,6.4,7.9,377.0,...,0.45,0.23,0.16,-320.0,-4.0,-183.0,18.2,18.2,0.0,2.0
2,1116,TW,38.0,52.6,3.7,3.3,2.7,6.7,8.5,403.0,...,0.5,0.25,0.26,136.0,0.0,-8.0,28.9,2.6,0.0,12.0
3,113,TR,22.0,54.5,3.1,2.8,4.5,10.9,5.4,346.0,...,0.26,0.3,0.23,297.0,-1.0,28.0,36.4,9.1,0.0,4.0
4,1Jiang,TW,44.0,59.1,3.5,3.9,2.4,4.3,8.6,414.0,...,0.41,0.2,0.2,172.0,4.0,114.0,22.7,13.6,0.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1584,yeat,TR,2.0,50.0,2.2,2.5,3.0,4.0,7.7,477.0,...,0.42,0.18,0.18,2097.0,26.0,1136.0,0.0,0.0,0.0,2.0
1585,zay,BR,16.0,25.0,2.8,2.1,3.3,7.4,1.3,260.0,...,2.26,0.64,0.5,86.0,-6.0,12.0,25.0,25.0,0.0,2.0
1586,zorenous,AU,30.0,53.3,2.6,2.3,3.1,5.9,7.7,387.0,...,0.39,0.2,0.12,22.0,3.0,6.0,13.3,16.7,0.0,10.0
1587,zs,CN,3.0,33.3,1.9,2.0,3.0,3.7,6.9,352.0,...,0,0,0,0,0,0,0,0,0,0


In [139]:
df_players.to_csv('S14_PLAYER_INFO.csv',index=False)